In [9]:
from textblob import TextBlob
import spacy
from spacy.lang.en import English
from spacy import displacy
import en_core_web_sm
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect 
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
print("Libraries Imported")

Libraries Imported


In [2]:
consumerKey = "cXOzZBpX78BW0HRmmS9hcDM5c"
consumerSecret = "55uqYYrGhw0Ca7QQhIUFH2t1K6k7IliYckfofUSCVhufSJ71H6"
accessToken = "1083286165845954560-LBqZLPApyhVsouDh8XzPoEQ1yPDWmO"
accessTokenSecret = "4bb8cDkRDH63lpYiNjIy4m9DIMH4hw9FktWR6X7gzCeBq"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit=True)
print("API Authentication")

API Authentication


In [10]:
nlp = English()

In [3]:
def tweet_cleaning(x): #remove link
    remove_RT = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x, count=1).strip())
    x = remove_RT(x)
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            x = x.replace(separator,' ')
    words = []
    for word in x.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [4]:
def tweet_sentiment_analysis(x):
    the_tweet = TextBlob(x)
    pol = the_tweet.sentiment.polarity
    sub = the_tweet.sentiment.subjectivity
    return pol, sub

In [11]:
def tweet_tokenizer(x) :
    text = x
    my_doc = nlp(text)
    tokens = []
    for token in my_doc :
        tokens.append(token.text)
    return tokens

In [5]:
keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = int(input ("Please enter how many tweets to analyze: "))
tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
tweet_list = []
for tweet in tweets :
    tweet_list.append(tweet.text)

Please enter keyword or hashtag to search:  biden
Please enter how many tweets to analyze:  100


In [6]:
cleaned_tweets_list = []
counter = 0
for x in tweet_list :
    cleaned_tweet = tweet_cleaning(x)
    print(counter+1,"-",cleaned_tweet)
    cleaned_tweets_list.append(cleaned_tweet)
    counter += 1

1 - We urged to prioritize appointment of Black women appellate judges since there was only one serving in entire coun…
2 - “They started calling me ‘Sleepy Tom ’ Why would they do that to me ” MVP quarterback Tom Brady trades zingers with Pres…
3 - woman DeSantis Calls Out Biden for Politically Motivated Double Standard on Cuban Immigrants “They have a 100X stronger ca…
4 - Tom Brady Joins President Biden to Laugh at Trump in the Worst Day of Trump’s Life https t co 9YiXVglRRu
5 - Joe Biden has LOST IT
6 - Patryku Jaki O co Biden ma się spierać z Ruskimi iNiemcami O bezpieczeństwo energetyczne Polski W i… https t co j1gHV4NP10
7 - regan Biden s Authoritarianism A Direct Assault on Our Freedom https t co eqgew4uNo6
8 - Retweet if you think Joe Biden has been a disaster for the USA
9 - BREAKING A Catholic university is hosting a town hall for Joe Biden even though he is radically pro abortion Call Mount…
10 - O ano de 2021 deu início a um novo marco para a condução da política econômica

In [7]:
counter = 0 
tweet_polarity = []
tweet_subjectivity = []
for x in cleaned_tweets_list :
    tweet_pol, tweet_sub = tweet_sentiment_analysis(x)
    print(counter+1,"\n","Tweet:",x,"\nPolarity:", tweet_pol,"\nSubjectivity",tweet_sub,"\n")
    counter += 1

1 
 Tweet: We urged to prioritize appointment of Black women appellate judges since there was only one serving in entire coun… 
Polarity: -0.05555555555555555 
Subjectivity 0.6861111111111112 

2 
 Tweet: “They started calling me ‘Sleepy Tom ’ Why would they do that to me ” MVP quarterback Tom Brady trades zingers with Pres… 
Polarity: 0.0 
Subjectivity 0.0 

3 
 Tweet: woman DeSantis Calls Out Biden for Politically Motivated Double Standard on Cuban Immigrants “They have a 100X stronger ca… 
Polarity: 0.0 
Subjectivity 0.03333333333333333 

4 
 Tweet: Tom Brady Joins President Biden to Laugh at Trump in the Worst Day of Trump’s Life https t co 9YiXVglRRu 
Polarity: -0.35 
Subjectivity 0.55 

5 
 Tweet: Joe Biden has LOST IT 
Polarity: 0.0 
Subjectivity 0.0 

6 
 Tweet: Patryku Jaki O co Biden ma się spierać z Ruskimi iNiemcami O bezpieczeństwo energetyczne Polski W i… https t co j1gHV4NP10 
Polarity: 0.0 
Subjectivity 0.0 

7 
 Tweet: regan Biden s Authoritarianism A Direct Assault on

In [12]:
token_list = []
index_list = []
counter = 0
for x in cleaned_tweets_list:
    tokens = tweet_tokenizer(x)
    token_list.append(tokens)
    counter += len(tokens)
    index_list.append(counter)

In [13]:
i = 0
for x in token_list and y in index_list :
    print(i+1,"\nWord:",x)
    i +=1

1 
Word: ['We', 'urged', 'to', 'prioritize', 'appointment', 'of', 'Black', 'women', 'appellate', 'judges', 'since', 'there', 'was', 'only', 'one', 'serving', 'in', 'entire', 'coun', '…']
2 
Word: ['“', 'They', 'started', 'calling', 'me', '‘', 'Sleepy', 'Tom', '’', 'Why', 'would', 'they', 'do', 'that', 'to', 'me', '”', 'MVP', 'quarterback', 'Tom', 'Brady', 'trades', 'zingers', 'with', 'Pres', '…']
3 
Word: ['woman', 'DeSantis', 'Calls', 'Out', 'Biden', 'for', 'Politically', 'Motivated', 'Double', 'Standard', 'on', 'Cuban', 'Immigrants', '“', 'They', 'have', 'a', '100X', 'stronger', 'ca', '…']
4 
Word: ['Tom', 'Brady', 'Joins', 'President', 'Biden', 'to', 'Laugh', 'at', 'Trump', 'in', 'the', 'Worst', 'Day', 'of', 'Trump', '’s', 'Life', 'https', 't', 'co', '9YiXVglRRu']
5 
Word: ['Joe', 'Biden', 'has', 'LOST', 'IT']
6 
Word: ['Patryku', 'Jaki', 'O', 'co', 'Biden', 'ma', 'się', 'spierać', 'z', 'Ruskimi', 'iNiemcami', 'O', 'bezpieczeństwo', 'energetyczne', 'Polski', 'W', 'i', '…', 'https', 